In [19]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5539,2020-11-13T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1450,210,1660,24626,...,1372,1707,11258,837,23824.0,14557.0,38381,799919,554557.0,NaN
5540,2020-11-13T17:00:00,ITA,9,Toscana,43.769231,11.255889,1656,265,1921,48208,...,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0,effettuati 40063 tamponi rapidi antigenici di...
5541,2020-11-13T17:00:00,ITA,10,Umbria,43.106758,12.388247,359,68,427,9963,...,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0,NaN
5542,2020-11-13T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,149,15,164,2199,...,25,103,2261,234,4295.0,563.0,4858,48853,29189.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5539,2020-11-13T17:00:00,19,Sicilia,1450,210,1660,24626,26286,1372,1707,11258,837,23824.0,14557.0,38381,799919,554557.0
5540,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0
5541,2020-11-13T17:00:00,10,Umbria,359,68,427,9963,10390,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0
5542,2020-11-13T17:00:00,2,Valle d'Aosta,149,15,164,2199,2363,25,103,2261,234,4295.0,563.0,4858,48853,29189.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-13', '2020-11-12')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-13T17:00:00,1,Piemonte,4890,353,5243,61411,66654,3456,5258,45694,4964,49229.0,68083.0,117312,1239594,785558.0
1,2020-11-13T17:00:00,2,Valle d'Aosta,149,15,164,2199,2363,25,103,2261,234,4295.0,563.0,4858,48853,29189.0
2,2020-11-13T17:00:00,3,Lombardia,7319,801,8120,143149,151269,8476,10634,134294,19028,231380.0,73211.0,304591,3476242,2115026.0
3,2020-11-13T17:00:00,5,Veneto,1831,219,2050,55238,57288,2533,3605,34454,2764,26878.0,67628.0,94506,2529005,978328.0
4,2020-11-13T17:00:00,6,Friuli Venezia Giulia,366,47,413,8738,9151,504,831,8114,506,15378.0,2393.0,17771,610398,250805.0
5,2020-11-13T17:00:00,7,Liguria,1372,110,1482,13403,14885,224,1209,24521,2058,28704.0,12760.0,41464,516330,261534.0
6,2020-11-13T17:00:00,8,Emilia-Romagna,2164,225,2389,45767,48156,2132,2384,29947,4965,54812.0,28256.0,83068,1824983,956466.0
7,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0
8,2020-11-13T17:00:00,10,Umbria,359,68,427,9963,10390,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0
9,2020-11-13T17:00:00,11,Marche,559,74,633,12201,12834,622,740,7902,1094,21500.0,330.0,21830,360800,211777.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-12T17:00:00,1,Piemonte,4833,348,5181,58017,63198,2844,4787,43953,4903,47320.0,64734.0,112054,1216816,770475.0
1,2020-11-12T17:00:00,2,Valle d'Aosta,159,16,175,2163,2338,6,167,2186,231,4201.0,554.0,4755,48023,28810.0
2,2020-11-12T17:00:00,3,Lombardia,7047,782,7829,134964,142793,875,9291,132254,18910,224343.0,69614.0,293957,3420606,2086645.0
3,2020-11-12T17:00:00,5,Veneto,1763,213,1976,52779,54755,2520,3564,33419,2727,26696.0,64205.0,90901,2511361,971907.0
4,2020-11-12T17:00:00,6,Friuli Venezia Giulia,346,47,393,8254,8647,482,838,7795,498,14675.0,2265.0,16940,603277,248809.0
5,2020-11-12T17:00:00,7,Liguria,1363,109,1472,13189,14661,706,1013,23554,2040,28067.0,12188.0,40255,509052,258621.0
6,2020-11-12T17:00:00,8,Emilia-Romagna,2094,228,2322,43702,46024,2133,2402,29735,4925,53583.0,27101.0,80684,1804471,947920.0
7,2020-11-12T17:00:00,9,Toscana,1653,256,1909,47007,48916,1016,1932,21198,1738,58012.0,13840.0,71852,1275580,830926.0
8,2020-11-12T17:00:00,10,Umbria,354,69,423,10361,10784,450,783,6026,226,4809.0,12227.0,17036,346677,197139.0
9,2020-11-12T17:00:00,11,Marche,536,74,610,11602,12212,698,834,7795,1083,20830.0,260.0,21090,357212,209704.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-13T17:00:00,1,Piemonte,4890,353,5243,61411,66654,3456,5258,45694,4964,49229.0,68083.0,117312,1239594,785558.0
1,2020-11-13T17:00:00,2,Valle d'Aosta,149,15,164,2199,2363,25,103,2261,234,4295.0,563.0,4858,48853,29189.0
2,2020-11-13T17:00:00,3,Lombardia,7319,801,8120,143149,151269,8476,10634,134294,19028,231380.0,73211.0,304591,3476242,2115026.0
3,2020-11-13T17:00:00,4,Trentino-Alto Adige,806,71,877,11757,12634,566,1022,14565,879,22530.0,5548.0,28078,593328,NaN
4,2020-11-13T17:00:00,5,Veneto,1831,219,2050,55238,57288,2533,3605,34454,2764,26878.0,67628.0,94506,2529005,978328.0
5,2020-11-13T17:00:00,6,Friuli Venezia Giulia,366,47,413,8738,9151,504,831,8114,506,15378.0,2393.0,17771,610398,250805.0
6,2020-11-13T17:00:00,7,Liguria,1372,110,1482,13403,14885,224,1209,24521,2058,28704.0,12760.0,41464,516330,261534.0
7,2020-11-13T17:00:00,8,Emilia-Romagna,2164,225,2389,45767,48156,2132,2384,29947,4965,54812.0,28256.0,83068,1824983,956466.0
8,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0
9,2020-11-13T17:00:00,10,Umbria,359,68,427,9963,10390,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-12T17:00:00,1,Piemonte,4833,348,5181,58017,63198,2844,4787,43953,4903,47320.0,64734.0,112054,1216816,770475.0
1,1,2020-11-12T17:00:00,2,Valle d'Aosta,159,16,175,2163,2338,6,167,2186,231,4201.0,554.0,4755,48023,28810.0
2,2,2020-11-12T17:00:00,3,Lombardia,7047,782,7829,134964,142793,875,9291,132254,18910,224343.0,69614.0,293957,3420606,2086645.0
3,19,2020-11-12T17:00:00,4,Trentino-Alto Adige,786,72,858,11210,12068,396,947,14130,858,21637.0,5419.0,27056,585495,NaN
4,3,2020-11-12T17:00:00,5,Veneto,1763,213,1976,52779,54755,2520,3564,33419,2727,26696.0,64205.0,90901,2511361,971907.0
5,4,2020-11-12T17:00:00,6,Friuli Venezia Giulia,346,47,393,8254,8647,482,838,7795,498,14675.0,2265.0,16940,603277,248809.0
6,5,2020-11-12T17:00:00,7,Liguria,1363,109,1472,13189,14661,706,1013,23554,2040,28067.0,12188.0,40255,509052,258621.0
7,6,2020-11-12T17:00:00,8,Emilia-Romagna,2094,228,2322,43702,46024,2133,2402,29735,4925,53583.0,27101.0,80684,1804471,947920.0
8,7,2020-11-12T17:00:00,9,Toscana,1653,256,1909,47007,48916,1016,1932,21198,1738,58012.0,13840.0,71852,1275580,830926.0
9,8,2020-11-12T17:00:00,10,Umbria,354,69,423,10361,10784,450,783,6026,226,4809.0,12227.0,17036,346677,197139.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-13T17:00:00,1,Piemonte,4890,353,5243,61411,66654,3456,5258,45694,4964,49229.0,68083.0,117312,1239594,785558.0,4341375
1,2020-11-13T17:00:00,2,Valle d'Aosta,149,15,164,2199,2363,25,103,2261,234,4295.0,563.0,4858,48853,29189.0,125501
2,2020-11-13T17:00:00,3,Lombardia,7319,801,8120,143149,151269,8476,10634,134294,19028,231380.0,73211.0,304591,3476242,2115026.0,10103969
3,2020-11-13T17:00:00,4,Trentino-Alto Adige,806,71,877,11757,12634,566,1022,14565,879,22530.0,5548.0,28078,593328,NaN,1074819
4,2020-11-13T17:00:00,5,Veneto,1831,219,2050,55238,57288,2533,3605,34454,2764,26878.0,67628.0,94506,2529005,978328.0,4907704
5,2020-11-13T17:00:00,6,Friuli Venezia Giulia,366,47,413,8738,9151,504,831,8114,506,15378.0,2393.0,17771,610398,250805.0,1211357
6,2020-11-13T17:00:00,7,Liguria,1372,110,1482,13403,14885,224,1209,24521,2058,28704.0,12760.0,41464,516330,261534.0,1543127
7,2020-11-13T17:00:00,8,Emilia-Romagna,2164,225,2389,45767,48156,2132,2384,29947,4965,54812.0,28256.0,83068,1824983,956466.0,4467118
8,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0,3722729
9,2020-11-13T17:00:00,10,Umbria,359,68,427,9963,10390,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-13T17:00:00,1,Piemonte,4890,353,5243,61411,66654,3456,5258,45694,4964,49229.0,68083.0,117312,1239594,785558.0,4341375,367
1,2020-11-13T17:00:00,2,Valle d'Aosta,149,15,164,2199,2363,25,103,2261,234,4295.0,563.0,4858,48853,29189.0,125501,20
2,2020-11-13T17:00:00,3,Lombardia,7319,801,8120,143149,151269,8476,10634,134294,19028,231380.0,73211.0,304591,3476242,2115026.0,10103969,983
3,2020-11-13T17:00:00,4,Trentino-Alto Adige,806,71,877,11757,12634,566,1022,14565,879,22530.0,5548.0,28078,593328,NaN,1074819,106
4,2020-11-13T17:00:00,5,Veneto,1831,219,2050,55238,57288,2533,3605,34454,2764,26878.0,67628.0,94506,2529005,978328.0,4907704,825
5,2020-11-13T17:00:00,6,Friuli Venezia Giulia,366,47,413,8738,9151,504,831,8114,506,15378.0,2393.0,17771,610398,250805.0,1211357,175
6,2020-11-13T17:00:00,7,Liguria,1372,110,1482,13403,14885,224,1209,24521,2058,28704.0,12760.0,41464,516330,261534.0,1543127,209
7,2020-11-13T17:00:00,8,Emilia-Romagna,2164,225,2389,45767,48156,2132,2384,29947,4965,54812.0,28256.0,83068,1824983,956466.0,4467118,516
8,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0,3722729,415
9,2020-11-13T17:00:00,10,Umbria,359,68,427,9963,10390,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0,880285,70


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-13T17:00:00,3,Lombardia,7319,801,8120,143149,151269,8476,10634,...,81.49,8185,1343,55636,28381.0,118,2040,19.11,0.10525,3.01
1,2020-11-13T17:00:00,1,Piemonte,4890,353,5243,61411,66654,3456,5258,...,96.19,3394,471,22778,15083.0,61,1741,23.08,0.12111,2.70
2,2020-11-13T17:00:00,15,Campania,2153,183,2336,80884,83220,3161,4079,...,42.86,2961,14,25510,19678.0,40,878,15.99,0.07050,1.80
3,2020-11-13T17:00:00,5,Veneto,1831,219,2050,55238,57288,2533,3605,...,26.55,2459,41,17644,6421.0,37,1035,20.43,0.07346,1.93
4,2020-11-13T17:00:00,12,Lazio,3018,260,3278,59121,62399,2480,2925,...,34.81,2414,239,28309,21816.0,34,411,10.33,0.04987,1.34
5,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,...,63.86,1201,546,18423,8687.0,55,1210,13.45,0.06656,2.00
6,2020-11-13T17:00:00,8,Emilia-Romagna,2164,225,2389,45767,48156,2132,2384,...,43.60,2065,-18,20512,8546.0,40,212,11.62,0.05337,1.86
7,2020-11-13T17:00:00,19,Sicilia,1450,210,1660,24626,26286,1372,1707,...,39.03,1308,15,10217,6343.0,35,300,16.71,0.03436,0.77
8,2020-11-13T17:00:00,16,Puglia,1249,158,1407,21104,22511,1256,1350,...,43.17,1187,-84,8461,6223.0,22,72,15.96,0.03368,0.79
9,2020-11-13T17:00:00,7,Liguria,1372,110,1482,13403,14885,224,1209,...,52.63,214,196,7278,2913.0,18,967,16.61,0.07835,2.69


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,68,2,70,97.14
1,Piemonte,353,14,367,96.19
2,Lombardia,801,182,983,81.49
3,Valle d'Aosta,15,5,20,75.00
4,Trentino-Alto Adige,71,35,106,66.98
5,Toscana,265,150,415,63.86
6,Marche,74,53,127,58.27
7,Liguria,110,99,209,52.63
8,Emilia-Romagna,225,291,516,43.60
9,Puglia,158,208,366,43.17


In [28]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [29]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
